# Decision threshold modification by Hardt et al. - Recruiting data

This notebook contains an implementation of the post-processing fairness intervention introduced in [Equality of opportunity in supervised learning](https://dl.acm.org/doi/10.5555/3157382.3157469) by Hardt et al. (2016) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360.

The intervention method achieves equalised odds as follows. If only the decisions are available, they randomly choose either the original decision or fixed outcome in a way that ensures agreement across both protected groups. If a score function is available they choose between two carefully chosen thresholds with a particular probability to ensure agreement of true and false positive rates.

This method can be proven to be the optimal postprocessing algorithm for Equalised Odds, however the randomness introduced into decision making - which in particular could mean two identical individuals receive different outcomes - might clash with intuitive notions of fairness.

In [2]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing.eq_odds_postprocessing import (
    EqOddsPostprocessing,
)
from helpers.fairness_measures import accuracy, equalised_odds_p

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it wi

In [3]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out hte preprocessing notebook for details on how this data was obtained.

Location of artifacts (model and data)

In [4]:
artifacts_dir = Path("../../../artifacts")

In [5]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [6]:
data_dir = artifacts_dir / "data" / "recruiting"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [7]:
train_sds = StandardDataset(
    train,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("race_white")

Define which binary value goes with the (un-)privileged group

In [8]:
privileged_groups = [{"race_white": 1.0}]
unprivileged_groups = [{"race_white": 0.0}]

## Load original model

For maximum reproducibility we can also load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [9]:
bl_model = joblib.load(artifacts_dir / "models" / "recruiting" / "baseline.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../../../../artifacts/models/recruiting/baseline.pkl'

Get prediction for validation and test data

In [ ]:
bl_val_pred = bl_model.predict(val.drop("salary", axis=1))
val_sds_pred = val_sds.copy(deepcopy=True)
val_sds_pred.labels = bl_val_pred.reshape(-1, 1)

bl_test_pred = bl_model.predict(test.drop("salary", axis=1))
test_sds_pred = test_sds.copy(deepcopy=True)
test_sds_pred.labels = bl_test_pred.reshape(-1, 1)

## Equalised odds
Given the original unfair model we apply Hardt et al.'s intervention to achieve equalised odds. More precisely, we learn the mitigation procedure based on the true and predicted labels of the validation data. The learning does not need any parameter tuning. We then apply the learnt procedure to the predictions of the test data and analyse the outcomes for fairness and accuracy.

Note that this intervention method does not allow for a continuous trade-off between fairness and accuracy. Instead, the output is a single combination of accuracy and fairness for the corrected predictions.

### Learn intervention

On validation data.

In [ ]:
# Learn parameters to equalize odds and apply to create a new dataset
eopp = EqOddsPostprocessing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    seed=np.random.seed(),
)
eopp = eopp.fit(val_sds, val_sds_pred)

### Apply intervention
On test data.

In [ ]:
test_sds_pred_tranf = eopp.predict(test_sds_pred)
test_sds_pred_tranf.scores = test_sds_pred_tranf.labels

### Analyse fairness and accuracy
On test data.

In [ ]:
fnr = np.abs(
    test_sds_pred_tranf.scores[(test.salary == 1) & (test.sex == 1)].mean()
    - test_sds_pred_tranf.scores[(test.salary == 1) & (test.sex == 0)].mean()
)
fpr = np.abs(
    test_sds_pred_tranf.scores[(test.salary == 0) & (test.sex == 1)].mean()
    - test_sds_pred_tranf.scores[(test.salary == 0) & (test.sex == 0)].mean()
)

In [ ]:
print(
    "Accuracy =", accuracy(test_sds_pred_tranf.scores.flatten(), test.salary)
)
print(
    "Female accuracy =",
    accuracy(
        test_sds_pred_tranf.scores.flatten()[test.sex == 0],
        test.salary[test.sex == 0],
    ),
)
print(
    "Male accuracy =",
    accuracy(
        test_sds_pred_tranf.scores.flatten()[test.sex == 1],
        test.salary[test.sex == 1],
    ),
)
print("FNR =", fnr)
print("FPR =", fpr)

In [ ]:
print(
    "Equalised odds = ",
    equalised_odds_p(
        test_sds_pred_tranf.scores.flatten(), test.sex, test.salary
    ),
)

### Plots

In [ ]:
eo_bar = go.Figure(
    data=[
        go.Bar(
            x=[label],
            y=[
                test_sds_pred_tranf.scores[
                    (test.sex == sex) & (test.salary == label)
                ].mean()
            ],
            name="Male" if sex else "Female",
        )
        for label in range(2)
        for sex in range(2)
    ]
)
eo_bar

In [ ]:
export_plot(eo_bar, "hardt-eo.json")